## The following functions must be either added to or replaced in the modified ST-Matching algorithm to create the STB-matching version 

### Behavioral Score Matrix Calculation

In [ ]:
def build_behavioral_score_matrix(all_candidates, edge_path_matrix, edges_drive, verbose=False):
    """
    Builds a behavioral score matrix using normalized_match_score from edges_drive.

    Parameters:
    - all_candidates: list of list of candidate points per GPS point
    - edge_path_matrix: matrix of shortest paths between each pair of candidates
    - edges_drive: GeoDataFrame of the network edges with normalized_match_score of each edge

    Returns:
    - behavioral_matrix: matrix of behavioral scores per candidate transition
    """

    behavioral_matrix = {}

    for i in range(1, len(all_candidates)):
        behavioral_matrix[i] = {}

        for j in range(len(all_candidates[i - 1])):  # previous point candidates
            behavioral_matrix[i][j] = {}

            for k in range(len(all_candidates[i])):  # current point candidates
                entry_score = all_candidates[i - 1][j].get('normalized_count')
                exit_score = all_candidates[i][k].get('normalized_count')

                internal_scores = []
                edge_path = edge_path_matrix[i][j][k]

                for u, v, key, _ in edge_path:
                    # Skip virtual nodes if any
                    if isinstance(u, tuple) or isinstance(v, tuple):
                        continue
                    
                    match = edges_drive[
                        (edges_drive['u'] == u) &
                        (edges_drive['v'] == v) &
                        (edges_drive['key'] == key)
                    ]
                    if not match.empty:
                        internal_score = match.iloc[0].get('normalized_count')
                        internal_scores.append(internal_score)

                # Combine scores: entry, internal, and exit
                total_scores = [entry_score] + internal_scores + [exit_score]

                if total_scores:
                    behavioral_score = np.mean(total_scores)
                else:
                    behavioral_score = 0.0

                behavioral_matrix[i][j][k] = behavioral_score

                if verbose:
                    print(f"Behavioral score P{i-1} cand {j} → P{i} cand {k}: {behavioral_score:.4f}")

    return behavioral_matrix

### Building the candidate graph considering the behavioral score

In [ ]:
def build_candidate_graph(all_candidates, score_matrix, temporal_matrix, behavioral_matrix):
    """
    Builds the candidate graph GT' as described in the ST-Matching paper.

    Parameters:
    - all_candidates: list of candidate lists per GPS point
    - score_matrix: spatial score matrix [i][j][k]
    - temporal_matrix: temporal similarity matrix [i][j][k]

    Returns:
    - G: a NetworkX DiGraph representing the candidate graph
    """
    G = nx.DiGraph()

    # Add all candidate nodes with observation probabilities
    for i, candidates in enumerate(all_candidates):
        for k, cand in enumerate(candidates):
            G.add_node((i, k), geometry=cand['projected_point'], obs_prob=cand['obs_prob'])

    # Add edges between candidates of consecutive points
    for i in range(1, len(all_candidates)):
        for j in range(len(all_candidates[i - 1])):
            for k in range(len(all_candidates[i])):
                spatial_score = score_matrix[i][j][k]
                temporal_score = temporal_matrix[i][j][k]
                behavioral_score = behavioral_matrix[i][j][k]
                final_score = spatial_score * temporal_score * behavioral_score

                if final_score > 0:
                    G.add_edge((i - 1, j), (i, k),
                               score=final_score,
                               trans_prob=spatial_score,
                               temp_sim=temporal_score,
                               behavior_score=behavioral_score,
                               weight=final_score)  # keep weight for networkx algorithms


    return G

### The Final STB Matching function which includes behavioral matrix score function

In [ ]:
def STB_Matching(sample, network, edges_gdf, verbose=False):
    #finding the candidates
    all_candidate_points = get_all_candidates(sample)
    
    #adding the observation probability to the candidates
    add_observation_probability(all_candidate_points)
    
    #spatial score
    score_matrix, edge_path = build_spatial_score_matrix(all_candidate_points, sample, network, verbose=verbose)
    
    #temporal score
    temporal_matrix = build_temporal_matrix(sample, edge_path, all_candidate_points, edges_drive, verbose=verbose)
    
    #behavioral score
    behavioral_matrix = build_behavioral_score_matrix(all_candidate_points, edge_path, edges_drive, verbose=verbose)
    
    #build candidate graph
    candidate_graph = build_candidate_graph(all_candidate_points, score_matrix, temporal_matrix, behavioral_matrix)
    
    #finding the best (longest) path
    best_candidate_path = find_best_path(candidate_graph)
    
    #getting the matched points and the final path 
    matched_points, matched_path = get_matched_points_and_edges(all_candidate_points, best_candidate_path, edge_path, edges_gdf, sample)
    
    return {
    'candidate_points': all_candidate_points,
    'score_matrix': score_matrix,
    'edge_path': edge_path,
    'temporal_matrix': temporal_matrix,
    'behavioral_matrix': behavioral_matrix,    
    'graph': candidate_graph,
    'best_path': best_candidate_path,
    'matched_points': matched_points,
    'matched_path': matched_path
    }